In [1]:
from haystack.dataclasses import Document
import pandas as pd

from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter

from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import GPTGenerator

from haystack.components.retrievers.in_memory import InMemoryBM25Retriever


from dotenv import load_dotenv
import os


/Users/macpro/anaconda3/envs/aihelperapi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(".env")
openai_key = os.getenv("OPENAI_KEY")

In [3]:
# Source https://www.kaggle.com/datasets/carrie1/ecommerce-data?resource=download
df = pd.read_csv("data.csv", encoding='latin1')

# Drop rows with empty CustomerID
df.dropna(subset=['CustomerID'], inplace=True)

# rename columns to all lower case
df.columns = [x.lower() for x in df.columns]

# Save df to dict
df_dict = df.to_dict("records")

In [5]:
haystack_documents = []

# Create a list of Haystack documents
for i in range(len(df_dict)):
    content_str = f"Name of item purchased: {df_dict[i]['description']} \
        \nQuantity purchased: {df_dict[i]['quantity']} \
        \nPrice of item: {df_dict[i]['unitprice']} \
        \nDate of purchase: {df_dict[i]['invoicedate']} \
        \nCountry of purchase: {df_dict[i]['country']} \
        \nCustomer ID: {df_dict[i]['customerid']} \
        \nInvoice Number: {df_dict[i]['invoiceno']} \
        \nStock Code: {df_dict[i]['stockcode']}" ,
    haystack_documents.append(Document(
        content=content_str[0],
        id = f"ZOOA{str(1000000 + i)}",
        meta={
            "invoiceno": df_dict[i]["invoiceno"],
            "stockcode": df_dict[i]["stockcode"],
            "description": df_dict[i]["description"],
            "quantity": df_dict[i]["quantity"],
            "invoicedate": df_dict[i]["invoicedate"],
            "unitprice": df_dict[i]["unitprice"],
            "customerid": df_dict[i]["customerid"],
            "country": df_dict[i]["country"],
        },
    ))

In [6]:
document_store = InMemoryDocumentStore(bm25_algorithm="BM25Plus")
document_store.write_documents(documents=haystack_documents)

406829

In [ ]:
document_store.filter_documents()[0]

In [42]:


######## Complete this section #############
prompt_template = """
You are an expert data analyst who helps customers and employees with their questions about purchases and products.
You use the information provided in the documents to answer the questions.
You can answer the following types of questions:
Questions regarding the order: please ask the user to give you the invoice number.
Questions regarding the product: please ask the user to give you the stock code.
Questions regarding purchases made on a given day: please ask the user to give you the date of purchase.
If you are asked to calculate the total price of a purchase, please ask the user to give you the invoice number and add the total price of the items in the purchase.
If you are asked to calculate the total number of items for a purchase, please ask the user to give you the invoice number and add the total number of items in the purchase.
If the documents do not contain the answer to the question, say that ‘Answer is unknown.’
Context:
{% for doc in documents %}
    Purchase information: {{ doc.content }} 
    Invoice Number: {{ doc.meta['invoiceno'] }} 
    Stock Code: {{doc.meta['stockcode']}}
    Quantity purchased: {{doc.meta['quantity']}}
    Date of purchase: {{doc.meta['invoicedate']}}
    Price per item: {{doc.meta['unitprice']}} \n
{% endfor %};
Question: {{query}}
\n Answer:
"""
prompt_builder = PromptBuilder(prompt_template)
retriever = InMemoryBM25Retriever(document_store=document_store)
############################################
llm = GPTGenerator(api_key=openai_key, 
                   generation_kwargs={"temperature": 0},
                   model='gpt-4')

prediction_pipeline = Pipeline()
prediction_pipeline.add_component("retriever", retriever)
prediction_pipeline.add_component("prompt_builder", prompt_builder)
prediction_pipeline.add_component("generator", llm)

prediction_pipeline.connect("retriever.documents", "prompt_builder.documents")
prediction_pipeline.connect("prompt_builder", "generator")


/Users/macpro/anaconda3/envs/aihelperapi/lib/python3.10/site-packages/haystack/core/component/component.py:120: UserWarning: GPTGenerator is deprecated and will be removed in the next beta release. Please use OpenAIGenerator instead.
  instance = super().__call__(*args, **kwargs)


In [28]:
query = "Items for order with invoice number 536365"
result = prediction_pipeline.run(data={"retriever": {"query": query}, 
                                       "prompt_builder": {"query": query},
                                       })
print(result['generator']['replies'][0])

Ranking by BM25...: 100%|██████████| 406829/406829 [00:06<00:00, 66506.74 docs/s]


The items for the order with invoice number 536365 are:
- WHITE METAL LANTERN (Stock Code: 71053)
- SET 7 BABUSHKA NESTING BOXES (Stock Code: 22752)
- WHITE HANGING HEART T-LIGHT HOLDER (Stock Code: 85123A)
- CREAM CUPID HEARTS COAT HANGER (Stock Code: 84406B)
- RED WOOLLY HOTTIE WHITE HEART (Stock Code: 84029E)
- GLASS STAR FROSTED T-LIGHT HOLDER (Stock Code: 21730)
- KNITTED UNION FLAG HOT WATER BOTTLE (Stock Code: 84029G)


In [43]:
query = "Total number of items purchased for order 536365"
result = prediction_pipeline.run(data={"retriever": {"query": query}, 
                                       "prompt_builder": {"query": query},
                                       })
print(result['generator']['replies'][0])

Ranking by BM25...: 100%|██████████| 406829/406829 [00:04<00:00, 83507.19 docs/s] 


The total number of items purchased for order 536365 is 6 (WHITE METAL LANTERN) + 2 (SET 7 BABUSHKA NESTING BOXES) + 6 (WHITE HANGING HEART T-LIGHT HOLDER) + 8 (CREAM CUPID HEARTS COAT HANGER) + 6 (RED WOOLLY HOTTIE WHITE HEART.) + 6 (GLASS STAR FROSTED T-LIGHT HOLDER) + 6 (KNITTED UNION FLAG HOT WATER BOTTLE) = 40 items.


In [44]:
query = "Total cost for order 536365"
result = prediction_pipeline.run(data={"retriever": {"query": query}, 
                                       "prompt_builder": {"query": query},
                                       })
print(result['generator']['replies'][0])

Ranking by BM25...: 100%|██████████| 406829/406829 [00:04<00:00, 84497.74 docs/s]


The total cost for order 536365 is calculated as follows:

WHITE METAL LANTERN: 6 * 3.39 = 20.34
SET 7 BABUSHKA NESTING BOXES: 2 * 7.65 = 15.3
WHITE HANGING HEART T-LIGHT HOLDER: 6 * 2.55 = 15.3
CREAM CUPID HEARTS COAT HANGER: 8 * 2.75 = 22
RED WOOLLY HOTTIE WHITE HEART: 6 * 3.39 = 20.34
GLASS STAR FROSTED T-LIGHT HOLDER: 6 * 4.25 = 25.5
KNITTED UNION FLAG HOT WATER BOTTLE: 6 * 3.39 = 20.34

Total cost = 20.34 + 15.3 + 15.3 + 22 + 20.34 + 25.5 + 20.34 = 139.12


In [33]:
df[df['invoiceno']=='536365']['quantity'].sum()

40

In [40]:
(df[df['invoiceno']=='536365']['quantity']*df[df['invoiceno']=='536365']['unitprice']).sum()

139.12

In [17]:
df[df['stockcode']=='84406B']['invoicedate'].unique()

array(['12/1/2010 8:26', '12/1/2010 9:02', '12/1/2010 9:32',
       '12/1/2010 10:51', '12/1/2010 11:33', '12/2/2010 9:41',
       '12/2/2010 9:44', '12/2/2010 10:54', '12/2/2010 10:56',
       '12/2/2010 11:41', '12/2/2010 12:22', '12/2/2010 12:23',
       '12/2/2010 12:25', '12/2/2010 14:04', '12/2/2010 14:06',
       '12/2/2010 15:24', '12/2/2010 15:26', '12/2/2010 17:41',
       '12/5/2010 13:05', '12/5/2010 13:08', '12/5/2010 13:18',
       '12/5/2010 13:49', '12/5/2010 13:55', '12/5/2010 16:24',
       '12/6/2010 11:26', '12/6/2010 14:36', '12/6/2010 15:27',
       '12/7/2010 13:29', '12/8/2010 10:35', '12/8/2010 13:03',
       '12/8/2010 16:15', '12/9/2010 13:34', '12/9/2010 16:53',
       '12/12/2010 13:26', '12/12/2010 16:14', '12/14/2010 11:48',
       '12/14/2010 14:40', '12/15/2010 9:58', '12/16/2010 18:20',
       '12/19/2010 13:31', '12/20/2010 11:56', '1/4/2011 12:58',
       '1/6/2011 15:25', '1/6/2011 16:03', '1/6/2011 18:57',
       '1/7/2011 13:17', '1/7/2011 16:32',

In [10]:

pipeline = Pipeline()
pipeline.add_component(instance=InMemoryBM25Retriever(document_store=document_store), name="retriever")

In [11]:
query = "Names of items purchased"
result = pipeline.run(data={"retriever": {"query": query, 
                                          "filters": {  "field": "meta.invoiceno", "operator": "==", "value": "536365"}}})

Ranking by BM25...: 100%|██████████| 7/7 [00:00<00:00, 53092.46 docs/s]


In [12]:
print(result)

{'retriever': {'documents': [Document(id=ZOOA1000001, content: 'Name of item purchased: WHITE METAL LANTERN         
Quantity purchased: 6         
Price of item: 3...', meta: {'invoiceno': '536365', 'stockcode': '71053', 'description': 'WHITE METAL LANTERN', 'quantity': 6, 'invoicedate': '12/1/2010 8:26', 'unitprice': 3.39, 'customerid': 17850.0, 'country': 'United Kingdom'}, score: 0.7061604193237588), Document(id=ZOOA1000005, content: 'Name of item purchased: SET 7 BABUSHKA NESTING BOXES         
Quantity purchased: 2         
Price o...', meta: {'invoiceno': '536365', 'stockcode': '22752', 'description': 'SET 7 BABUSHKA NESTING BOXES', 'quantity': 2, 'invoicedate': '12/1/2010 8:26', 'unitprice': 7.65, 'customerid': 17850.0, 'country': 'United Kingdom'}, score: 0.7029024403863381), Document(id=ZOOA1000006, content: 'Name of item purchased: GLASS STAR FROSTED T-LIGHT HOLDER         
Quantity purchased: 6         
Pr...', meta: {'invoiceno': '536365', 'stockcode': '21730', 'descriptio

In [ ]:
document_store.filter_documents(filters={"meta.invoiceno": ["536365"]})

In [ ]:
df[df['InvoiceNo']=='536365']